In [ ]:
# online_retail_eda.py
# EDA cho bộ dữ liệu "Online Retail" (Kaggle)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------
# Config
# -------------------------
XLSX_PATH = "../input/online-retail/Online Retail.xlsx"
OUTPUT_DIR = "figs_retail"
TABLE_DIR = "tables_retail"

def ensure_dir(p): os.makedirs(p, exist_ok=True)
ensure_dir(OUTPUT_DIR); ensure_dir(TABLE_DIR)

# -------------------------
# 1) Load
# -------------------------
try:
    df = pd.read_excel(XLSX_PATH, engine="openpyxl")
except Exception:
    # fallback nếu engine khác
    df = pd.read_excel(XLSX_PATH)

# Chuẩn tên cột (một số bản ghi có thể hơi khác hoa/thường)
df.columns = [str(c).strip() for c in df.columns]

# Kiểm tra cột tối thiểu
needed = ["InvoiceNo","StockCode","Description","Quantity","InvoiceDate","UnitPrice","CustomerID","Country"]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise ValueError(f"Thiếu cột: {missing}\nĐang có: {list(df.columns)}")

# -------------------------
# 2) Làm sạch dữ liệu
# -------------------------
# Bỏ NA các cột quan trọng
df = df.dropna(subset=["InvoiceNo","StockCode","Description","InvoiceDate","UnitPrice","Quantity","Country"]).copy()

# Ép kiểu
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df["UnitPrice"] = pd.to_numeric(df["UnitPrice"], errors="coerce")
# CustomerID đôi khi dạng float -> cast sang Int64 (NA-friendly)
df["CustomerID"] = pd.to_numeric(df["CustomerID"], errors="coerce").astype("Int64")
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

# Loại hoá đơn huỷ: InvoiceNo bắt đầu bằng 'C'
df = df[~df["InvoiceNo"].astype(str).str.startswith("C")].copy()

# Lọc Quantity>0 và UnitPrice>0
df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)].copy()

# Tạo chỉ tiêu Revenue
df["Revenue"] = df["Quantity"] * df["UnitPrice"]

# (tuỳ chọn) loại duplicate y hệt
before = len(df)
df.drop_duplicates(inplace=True)
after = len(df)
print(f"Removed {before - after} duplicate rows.")

# -------------------------
# 3) Tóm tắt nhanh
# -------------------------
print("\n== Shape sau làm sạch ==", df.shape)
print("\n== Missing values per column ==")
print(df[needed + ["Revenue"]].isna().sum())

print("\n== Mô tả thống kê (Quantity, UnitPrice, Revenue) ==")
desc = df[["Quantity","UnitPrice","Revenue"]].describe().T.round(3)
print(desc)
desc.to_csv(os.path.join(TABLE_DIR, "describe_basic.csv"))

# -------------------------
# 4) Top sản phẩm / quốc gia
# -------------------------
top_prod = (df.groupby("Description")["Revenue"]
              .sum().sort_values(ascending=False).head(10))
top_prod.to_csv(os.path.join(TABLE_DIR, "top10_products_by_revenue.csv"))

plt.figure(figsize=(12,5))
top_prod.plot(kind="bar")
plt.title("Top 10 Products by Revenue"); plt.ylabel("Revenue")
plt.tight_layout(); plt.savefig(os.path.join(OUTPUT_DIR, "top10_products_by_revenue.png"), dpi=140)
plt.close()

top_country = (df.groupby("Country")["Revenue"]
                 .sum().sort_values(ascending=False).head(10))
top_country.to_csv(os.path.join(TABLE_DIR, "top10_countries_by_revenue.csv"))

plt.figure(figsize=(12,5))
top_country.plot(kind="bar")
plt.title("Top 10 Countries by Revenue"); plt.ylabel("Revenue")
plt.tight_layout(); plt.savefig(os.path.join(OUTPUT_DIR, "top10_countries_by_revenue.png"), dpi=140)
plt.close()

# -------------------------
# 5) Phân tích theo thời gian
# -------------------------
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
df = df.dropna(subset=["InvoiceDate"]).copy()
df = df.set_index("InvoiceDate").sort_index()

# Doanh thu theo ngày
daily = df["Revenue"].resample("D").sum()
daily.to_csv(os.path.join(TABLE_DIR, "daily_revenue.csv"))
plt.figure(figsize=(14,4))
daily.plot()
plt.title("Daily Revenue"); plt.ylabel("Revenue")
plt.tight_layout(); plt.savefig(os.path.join(OUTPUT_DIR, "daily_revenue.png"), dpi=140)
plt.close()

# Theo giờ trong ngày (giờ bán chạy)
df["Hour"] = df.index.hour
hour_rev = df.groupby("Hour")["Revenue"].sum()
hour_rev.to_csv(os.path.join(TABLE_DIR, "revenue_by_hour.csv"))
plt.figure()
hour_rev.plot(kind="bar")
plt.title("Revenue by Hour of Day"); plt.xlabel("Hour"); plt.ylabel("Revenue")
plt.tight_layout(); plt.savefig(os.path.join(OUTPUT_DIR, "revenue_by_hour.png"), dpi=140)
plt.close()

# Theo tháng (seasonality)
month_rev = df["Revenue"].resample("M").sum()
month_rev.to_csv(os.path.join(TABLE_DIR, "monthly_revenue.csv"))
plt.figure(figsize=(12,4))
month_rev.plot()
plt.title("Monthly Revenue"); plt.ylabel("Revenue")
plt.tight_layout(); plt.savefig(os.path.join(OUTPUT_DIR, "monthly_revenue.png"), dpi=140)
plt.close()

# -------------------------
# 6) RFM cơ bản (Recency, Frequency, Monetary)
# -------------------------
# snapshot_date = 1 ngày sau thời điểm hoá đơn cuối
snapshot_date = df.index.max() + pd.Timedelta(days=1)

rfm = (df.reset_index()
         .dropna(subset=["CustomerID"])
         .groupby("CustomerID")
         .agg(
             Recency = ("InvoiceDate", lambda s: (snapshot_date - s.max()).days),
             Frequency = ("InvoiceNo", "nunique"),
             Monetary = ("Revenue", "sum")
         )
      )
rfm = rfm.sort_values("Monetary", ascending=False)
rfm.to_csv(os.path.join(TABLE_DIR, "rfm.csv"))

# Phân phối R/F/M
plt.figure(figsize=(12,4))
rfm["Recency"].plot(kind="hist", bins=40, edgecolor="black")
plt.title("Recency distribution"); plt.xlabel("days"); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "rfm_recency_hist.png"), dpi=140); plt.close()

plt.figure(figsize=(12,4))
rfm["Frequency"].plot(kind="hist", bins=40, edgecolor="black")
plt.title("Frequency distribution"); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "rfm_frequency_hist.png"), dpi=140); plt.close()

plt.figure(figsize=(12,4))
rfm["Monetary"].plot(kind="hist", bins=40, edgecolor="black")
plt.title("Monetary distribution"); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "rfm_monetary_hist.png"), dpi=140); plt.close()

# -------------------------
# 7) Top khách hàng theo doanh thu
# -------------------------
top_cust = rfm["Monetary"].head(20)
plt.figure(figsize=(12,5))
top_cust.plot(kind="bar")
plt.title("Top 20 Customers by Monetary"); plt.ylabel("Revenue")
plt.tight_layout(); plt.savefig(os.path.join(OUTPUT_DIR, "top20_customers_monetary.png"), dpi=140)
plt.close()

# -------------------------
# 8) Phân tích giỏ hàng (basket) cơ bản (tùy chọn, nhanh)
# - Xem top các cặp mặt hàng cùng xuất hiện trong 1 hoá đơn
# -------------------------
# Tạo "basket" theo InvoiceNo -> danh sách StockCode (hoặc Description)
basket = (df.reset_index()[["InvoiceNo","Description"]]
            .drop_duplicates()
            .groupby("InvoiceNo")["Description"].apply(list))

# Đếm cặp sản phẩm xuất hiện cùng nhau
from collections import Counter
pair_counter = Counter()
for items in basket:
    # lấy unique trong 1 invoice để tránh đếm lặp
    uniq = list(dict.fromkeys(items))
    n = len(uniq)
    for i in range(n):
        for j in range(i+1, n):
            a, b = sorted((uniq[i], uniq[j]))
            pair_counter[(a,b)] += 1

top_pairs = pair_counter.most_common(20)
top_pairs_df = pd.DataFrame(top_pairs, columns=["pair","count"])
top_pairs_df.to_csv(os.path.join(TABLE_DIR, "top20_pairs.csv"), index=False)

plt.figure(figsize=(12,6))
plt.barh([f"{a[:25]} | {b[:25]}" for (a,b),_ in top_pairs], [c for _,c in top_pairs][::-1])
plt.title("Top 20 product pairs (co-occurrence)")
plt.xlabel("count"); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "top20_product_pairs.png"), dpi=140)
plt.close()

print(f"\nSaved figures -> ./{OUTPUT_DIR}/")
print(f"Saved tables  -> ./{TABLE_DIR}/")


Removed 5226 duplicate rows.

== Shape sau làm sạch == (524878, 9)

== Missing values per column ==
InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     132186
Country             0
Revenue             0
dtype: int64

== Mô tả thống kê (Quantity, UnitPrice, Revenue) ==
              count    mean      std    min   25%   50%    75%        max
Quantity   524878.0  10.617  156.280  1.000  1.00  4.00  11.00   80995.00
UnitPrice  524878.0   3.923   36.093  0.001  1.25  2.08   4.13   13541.33
Revenue    524878.0  20.275  271.694  0.001  3.90  9.92  17.70  168469.60


/tmp/ipykernel_37/3065284162.py:127: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_rev = df["Revenue"].resample("M").sum()



Saved figures -> ./figs_retail/
Saved tables  -> ./tables_retail/
